In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/04 23:22:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.3.2'

In [4]:
!pip install pandas

In [6]:
import pandas as pd

url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz"
df = pd.read_csv(url, sep=',', index_col=0)

df.to_csv("fhv_tripdata_2019-10.csv")

In [7]:
df.head(5)

,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
dispatching_base_num,,,,,,
B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264.0,264.0,NaN,B00009
B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264.0,264.0,NaN,B00013
B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264.0,264.0,NaN,B00014
B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264.0,264.0,NaN,B00014
B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264.0,264.0,NaN,B00014


In [8]:
from pyspark.sql import types

In [54]:
schema = types.StructType([
types.StructField('dispatching_base_num', types.StringType(), True),
types.StructField('pickup_datetime', types.TimestampType(), True),
types.StructField('dropOff_datetime', types.TimestampType(), True),
types.StructField('PUlocationID', types.FloatType(), True),
types.StructField('DOlocationID', types.FloatType(), True),
types.StructField('SR_Flag', types.StringType(), True),
types.StructField('Affiliated_base_number', types.StringType(), True)]
)

In [63]:
spark_df = spark.read.option("header", "true").schema(schema).csv("fhv_tripdata_2019-10.csv")
spark_df = spark_df.withColumn("PUlocationID", F.col("PUlocationID").cast(types.IntegerType()))
spark_df = spark_df.withColumn("DOlocationID", F.col("DOlocationID").cast(types.IntegerType()))

In [64]:
spark_df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   null|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   null|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   null|                B00014|
|     B00021         |2019-10-01 00:00:4

In [13]:
spark_df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropOff_datetime', TimestampType(), True), StructField('PUlocationID', IntegerType(), True), StructField('DOlocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [15]:
spark_df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [16]:
spark_df = spark_df.repartition(6)

In [17]:
spark_df.write.parquet('fhvhv')

In [2]:
!ls -lat -h ./fhvhv

total 34M
drwxr-xr-x 1 root root 4.0K Mar  5 05:44 ..
drwxr-xr-x 2 root root 4.0K Mar  4 23:27 .
-rw-r--r-- 1 root root    8 Mar  4 23:27 ._SUCCESS.crc
-rw-r--r-- 1 root root    0 Mar  4 23:27 _SUCCESS
-rw-r--r-- 1 root root  44K Mar  4 23:27 .part-00002-327cbb13-c9ec-4a46-8f7d-7881294fbdcd-c000.snappy.parquet.crc
-rw-r--r-- 1 root root 5.5M Mar  4 23:27 part-00002-327cbb13-c9ec-4a46-8f7d-7881294fbdcd-c000.snappy.parquet
-rw-r--r-- 1 root root  44K Mar  4 23:27 .part-00000-327cbb13-c9ec-4a46-8f7d-7881294fbdcd-c000.snappy.parquet.crc
-rw-r--r-- 1 root root  44K Mar  4 23:27 .part-00001-327cbb13-c9ec-4a46-8f7d-7881294fbdcd-c000.snappy.parquet.crc
-rw-r--r-- 1 root root  44K Mar  4 23:27 .part-00003-327cbb13-c9ec-4a46-8f7d-7881294fbdcd-c000.snappy.parquet.crc
-rw-r--r-- 1 root root  44K Mar  4 23:27 .part-00004-327cbb13-c9ec-4a46-8f7d-7881294fbdcd-c000.snappy.parquet.crc
-rw-r--r-- 1 root root  44K Mar  4 23:27 .part-00005-327cbb13-c9ec-4a46-8f7d-7881294fbdcd-c000.snappy.parquet.crc
-rw-r

In [18]:
df = spark.read.parquet('fhvhv')

In [19]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [21]:
from pyspark.sql import functions as F
date_df = df.withColumn('pickup_datetime', F.to_date(df.pickup_datetime))
date_df.show()

+--------------------+---------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+---------------+-------------------+------------+------------+-------+----------------------+
|              B00608|     2019-10-02|2019-10-02 23:58:13|        null|        null|   null|                B00608|
|              B01730|     2019-10-02|2019-10-02 16:23:03|        null|        null|   null|                B02096|
|              B00821|     2019-10-04|2019-10-04 15:53:08|        null|        null|   null|                B03021|
|              B02920|     2019-10-03|2019-10-03 17:47:55|        null|        null|   null|                B01536|
|              B01196|     2019-10-03|2019-10-03 07:00:54|        null|        null|   null|                B01196|
|              B00256|     2019-10-01|2019-10-01 03:55:16|        null| 

In [27]:
date_df.select('*').filter(date_df.pickup_datetime == '2019-10-15').count()

62610

In [31]:
duration_df = spark_df.withColumn(
    "duration", 
    (F.datediff(F.col("dropoff_datetime"), F.col("pickup_datetime")) * 24)
)
result_df = duration_df.select("duration").orderBy(F.col("duration").desc())

result_df.show()

+--------+
|duration|
+--------+
|  631152|
|  631152|
|   87696|
|   70128|
|    8808|
|    8784|
|    1464|
|    1056|
|    1056|
|     792|
|     792|
|     792|
|     792|
|     792|
|     792|
|     792|
|     792|
|     792|
|     792|
|     792|
+--------+
only showing top 20 rows



In [49]:
spark_df = spark.read.option("header", "true").schema(schema).csv("fhv_tripdata_2019-10.csv")

In [51]:
spark_df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: short (nullable = true)
 |-- DOlocationID: short (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [53]:
spark_df.filter(spark_df.PUlocationID.isNotNull()).show()

+--------------------+---------------+----------------+------------+------------+-------+----------------------+
|dispatching_base_num|pickup_datetime|dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+---------------+----------------+------------+------------+-------+----------------------+
+--------------------+---------------+----------------+------------+------------+-------+----------------------+



In [35]:
pd_df = pd.read_csv('fhv_tripdata_2019-10.csv')

In [37]:
pd_df.isnull().sum()

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID                 8556
DOlocationID                 8179
SR_Flag                   1897493
Affiliated_base_number      16955
dtype: int64

In [84]:
zone_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv'
zone_df = pd.read_csv(zone_url, sep=',', index_col=0)
zone_df.to_csv('taxi_zone_loop.csv')
zone_df = spark.read.csv('taxi_zone_loop.csv', header=True, inferSchema=True)

In [85]:
print(type(zone_df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [88]:
zone_df.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [89]:
zone_df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [93]:
joined_df = spark_df.join(zone_df, F.col("PUlocationID") == F.col("LocationID"), 'left')
grouped_df = joined_df.groupBy(F.col("PULocationID"), F.col("Zone")).agg(F.count("*").alias("cnt"))
result_df = grouped_df.orderBy(F.col("cnt")).limit(1)
result_df.show()

+------------+-----------+---+
|PULocationID|       Zone|cnt|
+------------+-----------+---+
|           2|Jamaica Bay|  1|
+------------+-----------+---+

